# Data Analysis

Now that we've got clean data, let's start with some basic financial analysis.

First, let's load our CSV file into a DataFrame, covert our dates, set the index, and check for duplicated rows or missing values.

In [ ]:
import pandas as pd

#url = "https://raw.githubusercontent.com/ImperialCollegeLondon/efds-ta-python/main/data/AAPL_2021_clean.csv"
#df = pd.read_csv(url)

df = pd.read_csv("AAPL_2021_clean.csv")
df["Date"] = pd.to_datetime(df["Date"])
df.set_index("Date", inplace=True)

print("Duplicates:", df.duplicated().sum())
print("Missing:", df.isnull().sum())

## Returns

Returns refer to the gain or loss made on an initial investment, often expressed as a percentage. We can use the generic **percentage change** formula here:

$$ (price_{end} - price_{start}) / price_{start} $$

We can apply this to close prices to calculate the simple daily return:

$$ (close price_{today} - close price_{yesterday}) / close price_{yesterday} $$

When available, use the Adjusted Close price, which takes into account corporate actions (dividends for example).

In [ ]:
aug18_closing = df.loc["2021-08-18", "Adj Close"]
aug17_closing = df.loc["2021-08-17", "Adj Close"]

aug18_return = (aug18_closing - aug17_closing) / aug17_closing
print(aug18_return)

This simple daily return expresses a loss in value of 2.5%

If we wanted to use the above approach to calculate daily returns for each day in our data set, it would take a long time. Let's see how we can use pandas `pct_change()` to make this sort of work easy, by applying our percentage change formula one column at a time.

In [ ]:
# Create a new column and populate it with daily returns
df['Daily Return'] = df['Adj Close'].pct_change()
df

Notice how the first row in our data has a missing value **NaN** in the new daily return column. This is because our data doesn't have a close price for the day before it!

What to do with this missing value depends on what further analysis we want to do. If we want to carry out simple descriptive statistics like compute the mean, max, or standard deviation,  we can leave our missing value as NaN, because pandas will by default ignore NaNs when calculating these.

In [ ]:
print("Average daily return:", df["Daily Return"].mean())

print("Highest daily return:", df["Daily Return"].max())

print("Standard Deviation:", df["Daily Return"].std())

For more complex analyses though, we may want to drop or fill this value. Let's calculate cumulative returns for the period. Instead of comparing a given day with the day before it, cumulative returns compare a given day with the first day of the period, to indicate how our stock has performed since our initial investment.

We generally fill missing daily returns with a 0, which indicates no change with the day before.

Because we're doing cumulative multiplication, we'll add 1 to the adjusted closing price, so we can compound the return over time.

In [ ]:
df["Daily Return"] = df["Daily Return"].fillna(0)

df["Cumulative Return"] = (1 + df["Daily Return"]).cumprod()

df

### Exercise 4

Calculate the daily change in trading volume for this stock, and then calculate the mean change in trading volume over the period. Take the same approach we used for calculating daily returns, but consider volume instead of the adjusted close price.

Then determine the trend in AAPL's stock this year:

- Rising volume and increasing price might indicate a **bullish** trend (where the uptrend is backed by strong demand and could continue).

- Rising volume and decreasing price might indicate a **bearish** trend (where the downtrend is backed by strong selling pressure and could continue).

- Falling volume on price increase or decrease often indicate a trend is losing strength. It might suggest that momentum is waning and a price reversal is coming.

In [ ]:
## YOUR CODE GOES HERE

## Surges

Surges in price or trading volume can be helpful indicators for traders. We usually define a surge as an increase on the day before by an amount higher than some defined threshold. A common threshold is two standard deviations above the mean. Let's look at price surges.

In [16]:
# Find the mean return
mean_return = df["Daily Return"].mean()

# Define a threshold as two standard deviations above the mean
return_threshold = mean_return + (df["Daily Return"].std() * 2)

# Define a condition
condition = df["Daily Return"] > return_threshold

# Subset the dataframe where daily returns are higher than the threshold
df[condition]

,Open,High,Low,Close,Adj Close,Volume,Daily Return,Cumulative Return,20-day MA,200-day MA,20-day SD,colour
Date,,,,,,,,,,,,
2021-01-20,128.660004,132.490005,128.550003,132.029999,130.085037,104319500,0.032856,1.020246,NaN,NaN,NaN,green
2021-01-21,133.800003,139.669998,133.589996,136.869995,134.853760,120150900,0.036659,1.057646,NaN,NaN,NaN,green
2021-03-01,123.750000,127.930000,122.790001,127.790001,126.095642,116307900,0.053851,0.988957,129.563227,NaN,5.122339,green
2021-03-09,119.029999,122.059998,118.790001,121.089996,119.484489,129525800,0.040650,0.937107,125.263821,NaN,5.926654,green
2021-12-07,169.080002,171.580002,168.339996,171.179993,169.698044,120405400,0.035446,1.330927,156.785328,137.846085,6.733253,green


## Moving Averages

Moving averages are a different kind of indicator, one that smooths out small variations in trading data to give a better picture of the overall trend.

A Simple Moving Average (SMA) is one which averages out a price over a specific period. The average is "moving" because when a new day is considered in the period, the oldest date is discarded.

Moving averages can be *fast*, when they cover a short period, or *slow* when they consider a longer period. The longer the period, the more those small variations are smoothed out.

In [ ]:
# Calculate a fast, 20-Day Moving Average
df['20-day MA'] = df['Adj Close'].rolling(window=20).mean()

# Calculate a slow, 200-Day Moving Average
df['200-day MA'] = df['Adj Close'].rolling(window=200).mean()

## Volatility

Volatility looks at the degree of variance in a stock, and can be helpful for determining risk. Periods of high standard deviation indicate higher volatility and may suggest a riskier investment.


In [ ]:
# Calculate 20-Day Standard Deviation
df['20-day SD'] = df['Adj Close'].rolling(window=20).std()
df

## Other useful functions

Another useful package in data science is NumPy. It actually powers alot of the work that pandas does, so technically, you've already used it!

One useful function is `np.where()`, often used for populating columns with a signal or indicator, depending on if a condition is met. Let's create a column to colour code our trading days. Days will have a different colour depening on if the market closes higher (green) or lower (red) than the opening.

In [ ]:
import numpy as np

df["colour"] = np.where(df["Close"] > df["Open"], "green", "red")
df